In [ ]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import re
import disallowedTermCheck
import urlRegExChecker
import emailRegExChecker

In [ ]:
#parameters for deny list and url 
denyListURL = "denylist.csv"
notebookListURL ="notebooklist.csv"

In [ ]:

#csv function ( to be implemented later)
#online dataset function (to be implemented later)

In [ ]:

#notebook checker function 
def notebookChecker(denyListURL, notebookListURL):
    denyTerms = pd.read_csv('denylist.csv')
    deniedTerms = np.array(denyTerms)
    denyList = {}
    for terms in deniedTerms:
        denyList.update({terms[0]:terms[1]})
    # print (f"\n Here we check the terms and their corresponding weights {denyList} \n")    
    
    disallowedTerms = []
    for term in denyList.keys():
        disallowedTerms += [term]
    #print (f"\n here we assign the terms to be checked to a list {terms} \n")
    
    checkedNotebooks = {}
    
    notebookList = pd.read_csv('notebooklist.csv')
    uncheckedNotebooks = np.array(notebookList)
    # print (f"\n Here we open and read the notebooks due to be open {uncheckedNotebooks} \n")
    

    #loop through files from list
    for uncheckedNotebook in uncheckedNotebooks:
        notebook = {}
        urlsFound = 0
        emailsFound = 0
        currentNotebook = open(str(f"notebookSamples/{uncheckedNotebook[0]}"), "r")
        # here we check line by line the the terms found which are added to a list 
        disallowedTermCount = {term : 0 for term in disallowedTerms}
        
        for line in currentNotebook:
            #loop trough line by line
            emailsFound += emailRegExChecker.emailChecker(line) 
            if urlRegExChecker.urlChecker(line) == True:
                urlsFound += 1

            checkedLine = disallowedTermCheck.termCheck(disallowedTerms, line)
            for term in checkedLine:
                if checkedLine[term] > 0:
                    disallowedTermCount[term] += 1
                    notebook.update({term: int(disallowedTermCount[term]) * int(denyList[term])}) 
                    checkedNotebooks.update({str(uncheckedNotebook): notebook})
        checkedNotebooks[str(uncheckedNotebook)]['urls'] = urlsFound
        checkedNotebooks[str(uncheckedNotebook)]['emails'] = emailsFound

    return checkedNotebooks 
    
    
notebookResults = notebookChecker(denyListURL, notebookListURL)
print(notebookResults)

In [ ]:
totals = {}

for nb in notebookResults:
    results = 0
    for term in notebookResults[nb]:
        results += notebookResults[nb][term]
    nb = nb.rsplit('.', 1)[0]
    nb = nb.rsplit('\'', 1)[1]
    totals[nb] = results
print(totals)
orderedResults = {term : score for term,score in sorted(totals.items(), key=lambda item:item[1])}

#PLot examples
f = plt.figure()
f.set_figwidth(3)
f.set_figheight(5)
plt.bar(range(len(orderedResults)),list(orderedResults.values()),align='center',color=['r','g','b'])
plt.xticks(range(len(orderedResults)),list(orderedResults.keys()))
plt.xlabel('Notebook Names')
plt.ylabel('Score')
plt.show

In [ ]:
for result in notebookResults:
    terms = notebookResults[result]
    g = plt.figure()
    g.set_figwidth(3)
    g.set_figheight(5)
    plt.bar(range(len(terms)),list(terms.values()),align='center',color=['r','g','b'])
    plt.xticks(range(len(terms)),list(terms.keys()))
    plt.xlabel(f" {result}")
    plt.ylabel('Score')
    plt.show

In [ ]:
#print in table
print('Notebook, terms found')
for k,v in notebookResults.items():
    print(k,v)

In [ ]:
df = pd.DataFrame.from_dict(notebookResults, orient='index')
print('terms found:')
print(df)

In [ ]:
#do we add open CSV function
#do we add open dataset function 